# Perception Test
    - Take RGBD images through camera
    - Total scene reconstruction based on aquired RGBD images
    - Segmentation objects of interest using cascade mask rcnn(based on swin-trasformer)
    - Get point cloud of object by taking overlapped part between segemtation and total reconstruction result

In [1]:
import open3d as o3d
import numpy as np
import pyrealsense2 as rs
import cv2
import os
import sys
import shutil
import json

## Directory, path setting

In [2]:
from utils.perception_config import *
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))
sys.path.append(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src/scripts/perception'))

cfg_off = '--src_path='
color_off = '--color_path='
depth_off = '--depth_path='
img_num_off = '--img_num='
output_off = '--output_path='

In [3]:
from scripts.perception.utils.function_utils import *

## Acquire RGBD images

In [3]:
streaming(DATASET_DIR)

/home/jhkim/Projects/rnb-planning/src/scripts/perception/dataset not empty. Overwrite? (y/n) : ㅜ'


## Scene Reconstruction
#### Issue : Not implemented yet. Further development is needed.

In [1]:
from boost_reconstruction.reconstruction_interface import reconstruction_interface_py as rci

In [5]:
rci.getReconstruction(COLOR_PATH, DEPTH_PATH, INTRINSIC_PATH, WORKING_DIR)

'========= Complete 3D Reconstruction ========='

In [6]:
# Construct Camera_parameter Class
# It has intrinsic, extrinsic parameter
cam_params = Camera_parameter(INTRINSIC_PATH, EXTRINSIC_PATH)

In [7]:
img_num = len(cam_params.cam_traj)

# Check img_num
print(img_num)

198


## Segmentation 

In [8]:
import subprocess

In [9]:
cfg_path_arg = cfg_off + WORKING_DIR + "/utils"
color_path_arg = color_off + COLOR_PATH
depth_path_arg = depth_off + DEPTH_PATH
img_num_arg = img_num_off + str(img_num)
output_path_arg = output_off + DATASET_DIR

In [10]:
_ = subprocess.check_output(['python3',
                    WORKING_DIR + '/utils' + '/object_detect.py',
                    cfg_path_arg, color_path_arg, depth_path_arg, img_num_arg, output_path_arg])

In [11]:
obj_detected_num = len(os.listdir(DATASET_DIR + '/color_segmented'))

# Check obj_detected_num
print(obj_detected_num)

5


## Get PCD of each objects
#### Issue : Poor pcd result of each object is obtained due to poor total Reconsturction

In [12]:
# Extract Overlapping points
obj_pcd_num = getPCDAll(img_num, DEPTH_SEG_PATH, RECONST_PCD_PATH, cam_params)

In [13]:
# Check obj_pcd_num
print(obj_pcd_num)

3


## Apply GPD
#### To get successful result, input point cloud of object should be dense, clear and sharp

In [14]:
import SharedArray as sa

In [15]:
from boost_gpd.gpd_interface import grasp_interface_py as gi

In [16]:
attacth_to_server()

In [21]:
grasp_list = getGPDAll(obj_pcd_num)

# Check grasp_list
print(len(grasp_list))

3


In [ ]:
grasp_list[0][0][:,:] = np.zeros((4,4))
grasp_list[1][0][:,:] = np.zeros((4,4))
grasp_list[2][0][:,:] = np.zeros((4,4))

In [27]:
# Set y-axis outward to match at gripper0 coordinate
for i in range(len(grasp_list)):
    for j in range(len(grasp_list[i])):
        grasp_list[i][j][:3,:3] = np.matmul(grasp_list[i][j][:3,:3], Rot_axis(3, np.pi/2))

In [28]:
# grasp_list[0][0][:,:] = np.zeros((4,4))
# grasp_list[1][0][:,:] = np.zeros((4,4))
# grasp_list[2][0][:,:] = np.zeros((4,4))
grasp_list[0][0][:,:] = np.identity(4)
# grasp_list[0][0][:3,:3] = Rot_axis(3, np.pi/4)
# grasp_list[0][1][:,:] = np.identity(4)
# grasp_list[0][1][:3,:3] = Rot_axis(3, np.pi/4)

## Simulate Robot motion

In [30]:
from pkg.controller.combined_robot import *
from pkg.project_config import *

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.indy7gripper, None,
                INDY_IP)]
              , connection_list=[False])

connection command:
indy0: False


In [31]:
from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(None)
# s_builder.reset_reference_coord(ref_name="floor")

In [32]:
# xyz_rpy_robots = s_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
xyz_rpy_robots = {"indy0": ((0,0,0), (0,0,-np.pi/2))}
crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = s_builder.create_gscene(crob)

Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0]


In [33]:
# # geometry 추가
# from pkg.geometry.geometry import *
# mobile_base = gscene.create_safe(gtype=GEOTYPE.BOX, name="mobile_base", link_name="base_link", 
#                    dims=(0.6,0.4,0.439), center=(0,0,-0.439/2), rpy=(0,0,0), 
#                    color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=False)
# floor = gscene.create_safe(gtype=GEOTYPE.BOX, name="floor", link_name="base_link", 
#                    dims=(10,10,0.01), center=(0,0,-0.439), rpy=(0,0,0), 
#                    color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=False)

In [34]:
gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)

Please create a subscriber to the marker


In [35]:
for gtem in gscene:
    round_it_str(gtem.dims)
    round_it_str(gtem.center)

In [36]:
# Add cam
from pkg.geometry.geotype import GEOTYPE
gscene.create_safe(gtype=GEOTYPE.CYLINDER, name="cam", link_name="indy0_tcp", 
                   dims=(0.061,0.061,0.026), center=(-0.0785,0,0.013), rpy=(0,0,0), 
                   color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=False)

gscene.create_safe(gtype=GEOTYPE.CYLINDER, name="cam_col", link_name="indy0_tcp", 
                   dims=(0.081,0.081,0.046), center=(-0.0785,0,0.013), rpy=(0,0,0), 
                   color=(0.0,1,0,0.3), display=True, fixed=True, collision=True)

viewpoint = gscene.create_safe(gtype=GEOTYPE.SPHERE, name="viewpoint", link_name="indy0_tcp", 
                   dims=(0.01,0.01,0.01), center=(-0.014,0,0), rpy=(0,0,-np.pi/2), 
                   color=(1,0,0,0.3), display=True, fixed=True, collision=False, parent="cam")

gscene.create_safe(gtype=GEOTYPE.CYLINDER, name="body", link_name="indy0_tcp", 
                   dims=(0.067,0.067,0.0335), center=(-0.0785,0,-0.01675), rpy=(0,0,0), 
                   color=(0.8,0.8,0.8,1), display=True, fixed=True, collision=False)

gscene.create_safe(gtype=GEOTYPE.CYLINDER, name="body_col", link_name="indy0_tcp", 
                   dims=(0.087,0.087,0.0535), center=(-0.0785,0,-0.01675), rpy=(0,0,0), 
                   color=(0.0,1,0,0.3), display=True, fixed=True, collision=True)

gscene.create_safe(gtype=GEOTYPE.SPHERE, name="backhead", link_name="indy0_tcp", 
                   dims=(0.067,0.067,0.067), center=(-0.0785,0,-0.0335), rpy=(0,0,0), 
                   color=(0.8,0.8,0.8,1), display=True, fixed=True, collision=False)

gscene.create_safe(gtype=GEOTYPE.SPHERE, name="backhead_col", link_name="indy0_tcp", 
                   dims=(0.087,0.087,0.087), center=(-0.0785,0,-0.0335), rpy=(0,0,0), 
                   color=(0.0,1,0,0.3), display=True, fixed=True, collision=True)

In [37]:
# INIT_POSE = np.deg2rad([  0., -28.,  85.,  -0.,  57., -180])
INIT_POSE = np.deg2rad([  -90., -22.,  70.,  0.,  60., -180.])
gscene.show_pose(INIT_POSE)

## Convert point cloud to Scene Geometry
#### Issue : Quite time consuming.  Complicated shape is difficult to decompose to simple geometry using ransac fitting

In [4]:
obj_pcd_num = 1

In [5]:
# Test box shape
box = o3d.geometry.TriangleMesh.create_box(width=0.06, height=0.06, depth=0.06)
R = box.get_rotation_matrix_from_xyz((5*np.pi / 8, 0, np.pi / 2))
box.rotate(R)
box.translate((0.18,0.42,0.38))
pcd = box.sample_points_uniformly(8000)
# pcd = pcd.uniform_down_sample(every_k_points=10)
pcd = pcd.voxel_down_sample(voxel_size=0.008)
# points = np.asarray(pcd.points)
# points += np.random.normal(0, 0.005, size=points.shape)
# pcd.points = o3d.utility.Vector3dVector(points)

In [36]:
# Test sphere shape
sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.05)
R = sphere.get_rotation_matrix_from_xyz((np.pi / 4, -np.pi / 4, np.pi / 4))
sphere.rotate(R)
sphere.translate((0.19,0.42,0.38))
pcd = sphere.sample_points_uniformly(6000)
# pcd = pcd.uniform_down_sample(every_k_points=10)
pcd = pcd.voxel_down_sample(voxel_size=0.01)

In [15]:
# Test cylinder shape
cyl = o3d.geometry.TriangleMesh.create_cylinder(radius=0.04, height=0.13)
R = cyl.get_rotation_matrix_from_xyz((5*np.pi / 8, 0, np.pi / 2))
cyl.rotate(R)
cyl.translate((0.19,0.395,0.38))
pcd = cyl.sample_points_uniformly(2200)
# pcd = pcd.uniform_down_sample(every_k_points=10)
pcd = pcd.voxel_down_sample(voxel_size=0.01)

In [16]:
pcd.paint_uniform_color((0,0,0))
o3d.visualization.draw_geometries([pcd])

In [10]:
obj_pcd_list = []
for i in range(obj_pcd_num):
    obj_pcd_list.append(o3d.io.read_point_cloud(WORKING_DIR + "/object_{}.pcd".format(i)))

In [6]:
for i in range(obj_pcd_num):
#     pcd = o3d.io.read_point_cloud(WORKING_DIR + '/krylon.pcd')
    pcd = o3d.io.read_point_cloud(WORKING_DIR + '/nontextured.ply')
#     pcd = o3d.io.read_point_cloud(WORKING_DIR + '/vase.ply').uniform_down_sample(every_k_points=20)
#     pcd = o3d.io.read_point_cloud(WORKING_DIR + '/cube.ply').uniform_down_sample(every_k_points=130)
#     pcd = o3d.io.read_point_cloud(WORKING_DIR + '/sphere.ply').uniform_down_sample(every_k_points=130)
    pcd = pcd.voxel_down_sample(voxel_size=0.01)
    p_inliers = geometry_matching(np.asarray(pcd.points), i, voxel_size=0.01, outliers_ratio_max=0.2,
                                  thres_ratioB=0.17, thres_ratioS=0.11, thres_ratioC=0.20)
#     p_inliers = geometry_matching(np.asarray(obj_pcd_list[i].uniform_down_sample(every_k_points=11).points),i)
    result = o3d.geometry.PointCloud()
    pcd.paint_uniform_color((0,0,0))
    result.points = o3d.utility.Vector3dVector(p_inliers)
    result.paint_uniform_color((1,0,0))
    o3d.visualization.draw_geometries([pcd, result])
#     o3d.visualization.draw_geometries([obj_pcd_list[i].uniform_down_sample(every_k_points=11), result])

('Total number :', 580)
('Cuboid inlier : ', 290)
('Sphere inlier : ', 153)
('Cylinder inlier : ', 207)
('Outlier ratio : ', 0.0)
Cuboid
[array([-0.00808687, -0.01841253,  0.08832682]), array([[-0.9989,  0.0196, -0.0417,  0.0152],
       [-0.0203, -0.9997,  0.0169,  0.0262],
       [-0.0414,  0.0177,  0.999 , -0.1757]]), array([[-9.991e-01,  3.820e-02, -1.670e-02, -2.650e-02],
       [ 3.830e-02,  9.993e-01, -2.900e-03,  6.250e-02],
       [ 1.660e-02, -3.600e-03, -9.999e-01,  1.000e-04]])]


In [7]:
def rendering_ransac_result(T_co, dims, geo_type):
    if (geo_type == 'Cuboid'):
        fitting_model = o3d.geometry.TriangleMesh.create_box(width=dims[0], height=dims[1], depth=dims[2])
        fitting_model.translate(T_co[:3,3] - (dims[0]/2.0, dims[1]/2.0, dims[2]/2.0))
        fitting_model.rotate(T_co[:3,:3])
        return fitting_model
    elif (geo_type == 'Sphere'):
        fitting_model = o3d.geometry.TriangleMesh.create_sphere(radius=dims[0])
        fitting_model.translate(T_co[:3,3])
        fitting_model.rotate(T_co[:3,:3])
        return fitting_model
    elif (geo_type == 'Cylinder'):
        fitting_model = o3d.geometry.TriangleMesh.create_cylinder(radius=dims[0], height=dims[2])
        fitting_model.translate(T_co[:3,3])
        fitting_model.rotate(T_co[:3,:3])
        return fitting_model  

In [8]:
# load해서 geometry 추가

total_geometry = []
for i in range(obj_pcd_num):
    with open(WORKING_DIR + '/obj_geometry_{}.json'.format(i), 'r') as f:
        json_data = json.load(f)
    component_num = len(json_data)
    fitting_model = []
    for j in range(component_num):
        # For only visualize inliers with original point cloud
        T_co, dims, geo_type = convertGeometry(json_data, j+1)
        fitting_model.append(rendering_ransac_result(T_co, dims, geo_type))
        pcd.paint_uniform_color((0,0,0))
    
    visualize_result = [pcd, result]
#     visualize_result = [obj_pcd_list[i].uniform_down_sample(every_k_points=11), result]
    for k in range(len(fitting_model)):
        visualize_result.append(fitting_model[k])
    o3d.visualization.draw_geometries(visualize_result)
#     o3d.visualization.draw_geometries([pcd, result, fitting_model[0]])


#     # Add geometry
# #     Qcur = np.deg2rad([  -90., -22.,  70.,  0.,  60., -180.])
#     T_bc = viewpoint.get_tf(list2dict(INIT_POSE, gscene.joint_names))
#     total_geometry.append(add_geometry(gscene, json_data, T_bc, component_num, i+1, grasp_list))

## Planning Scene

In [44]:
from pkg.planning.scene import PlanningScene

In [45]:
pscene = PlanningScene(combined_robot=crob, gscene=gscene)

In [46]:
# Add grip point
gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip0", link_name="indy0_tcp",
                   dims=(0.01,)*3, center=(0, 0, 0.14), rpy=(-np.pi/2,0,0), 
                   color=(1,0,0,1), display=True, collision=False, fixed=True)

In [47]:
from pkg.planning.constraint.constraint_actor import Gripper2Tool, PlacePlane
pscene.create_binder(bname="grip0", gname="grip0", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))

In [48]:
table = gscene.create_safe(gtype=GEOTYPE.BOX, name="table", link_name="base_link",
                           dims=(0.5,0.5,0.4), center=(0.6, 0.2, 0.2), rpy=(0,0,0),
                           color=(0.7,0.7,0.7,1), display=True, collision=True, fixed=True)
pscene.create_binder(bname="table", gname="table", _type=PlacePlane, point=None)

In [49]:
goal = gscene.create_safe(gtype=GEOTYPE.BOX, name="goal", link_name="base_link",
                           dims=(0.1,0.1,0.01), center=(0.5, 0.4, 0.395), rpy=(0,0,0),
                           color=(0.7,0.0,0.0,1), display=True, collision=True, fixed=True)
pscene.create_binder(bname="goal", gname="goal", _type=PlacePlane, point=None)

# Planning pipeline

In [50]:
from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

from pkg.ui.ui_broker import *

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
generate table - Handle
generate table - Bindergenerate table - Geometry

generate table - Object


In [77]:
# # example cup
# cup_cyl = gscene.create_safe(gtype=GEOTYPE.CYLINDER, name="cup_cyl", link_name="base_link",
#                    dims=(0.05,0.05, 0.07), center=(0.5, 0.2, 0.436), rpy=(0,0,0),
#                    color=(0.7,0.7,0.3,1), display=True, collision=True, fixed=False)
# cup_handle = gscene.create_safe(gtype=GEOTYPE.BOX, name="cup_handle", link_name="base_link",
#                    dims=(0.02,0.02, 0.05), center=(0.035, 0, 0), rpy=(0,0,0),
#                    color=(0.7,0.7,0.3,1), display=True, collision=True, fixed=False, parent="cup_cyl")

# from pkg.planning.constraint.constraint_subject import CustomObject, PlacePoint, DirectedPoint, ConstraintType
# ##
# # @class Grasp2PointFixed
# # @brief ActionPoint for rnb-planning.src.pkg.planning.constraint.constraint_actor.Gripper2Tool
# class Grasp2PointFixed(DirectedPoint):
#     ctype=ConstraintType.Grasp2
#     def get_redundancy(self):
#         return {}
# action_points_dict = {"cup_placement": PlacePoint("cup_placement", cup_cyl, 
#                                                         point=[0,0,-cup_cyl.dims[2]/2-0.001], 
#                                                         rpy=[0,0,0])}
# action_points_dict.update({"cup_grasp": Grasp2PointFixed("cup_grasp", cup_cyl, 
#                                                     point=(0,0,0), 
#                                                     rpy=Rot2rpy(np.matmul(Rot_rpy((0,-np.pi/2,np.pi/2)), Rot_axis(3, -np.pi/4))))})
# obj_pscene = pscene.create_subject(oname="cup", gname="cup_cyl", _type=CustomObject, 
#                              action_points_dict=action_points_dict)

generate table - Handle
generate table - Binder
generate table - Geometry
generate table - Object


## Add Grasping point from result of GPD

In [51]:
from pkg.planning.constraint.constraint_subject import CustomObject, PlacePoint, DirectedPoint, ConstraintType
##
# @class Grasp2PointFixed
# @brief ActionPoint for rnb-planning.src.pkg.planning.constraint.constraint_actor.Gripper2Tool
class Grasp2PointFixed(DirectedPoint):
    ctype=ConstraintType.Grasp2
    def get_redundancy(self):
        return {}

for i in range(obj_pcd_num):
    action_points_dict = {"obj_{}_placement".format(i+1): PlacePoint("obj_{}_placement".format(i+1), total_geometry[i], 
                                                point=[0,0,-total_geometry[i].dims[2]/2-0.001], rpy=[0,0,0])}
    for j in range(len(grasp_list[i])):
        if (np.array_equal(grasp_list[i][j], np.zeros((4,4))) == False):
            action_points_dict.update({"obj_{}".format(i+1) + "_grasp_{}".format(j+1): Grasp2PointFixed("obj_{}".format(i+1) + "_grasp_{}".format(j+1), total_geometry[i], 
                                                point=(0,0,0), rpy=Rot2rpy(np.matmul(np.matmul(grasp_list[i][j][:3,:3], Rot_axis(1, np.pi)), Rot_axis(2, -np.pi/2))))})
    
    obj_pscene = pscene.create_subject(oname="obj_pscene_{}".format(i+1), gname="obj_{}".format(i+1),
                                       _type=CustomObject, action_points_dict=action_points_dict)

generate table - Handle
generate table - Binder
generate table - Geometry
generate table - Object


In [52]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner

In [53]:
mplan = MoveitPlanner(pscene)
ppline.set_motion_planner(mplan=mplan)

In [54]:
from pkg.planning.task.rrt import TaskRRT

In [55]:
tplan = TaskRRT(pscene)
ppline.set_task_planner(tplan)

In [56]:
initial_state = pscene.initialize_state(INIT_POSE)
# initial_state = pscene.initialize_state(crob.home_pose)

In [57]:
goal_nodes = [('goal',)]

In [58]:
ppline.search(initial_state=initial_state, goal_nodes=goal_nodes, 
              verbose=True, add_homing=True, 
              timeout=1, timeout_loop=10)

try: 0 - ('table',)->('grip0',)
try transition motion
transition motion tried: True
result: 0 - ('table',)->('grip0',) = success
branching: 0->1 (0.08/10.0 s, steps/err: 70(75.0870704651 ms)/0.00221066414226)
try: 1 - ('grip0',)->('goal',)
try transition motion
transition motion tried: False
Motion Plan Failure
result: 1 - ('grip0',)->('goal',) = fail
try: 1 - ('grip0',)->('table',)
try transition motion
transition motion tried: False
Motion Plan Failure
result: 1 - ('grip0',)->('table',) = fail
try: 1 - ('grip0',)->('table',)
try transition motion
transition motion tried: True
result: 1 - ('grip0',)->('table',) = success
branching: 1->2 (2.19/10.0 s, steps/err: 15(55.5620193481 ms)/0.00170784214308)
try: 0 - ('table',)->('grip0',)
try transition motion
transition motion tried: True
result: 0 - ('table',)->('grip0',) = success
branching: 0->3 (2.31/10.0 s, steps/err: 93(114.939928055 ms)/0.00109688785466)
try: 3 - ('grip0',)->('goal',)
try transition motion
transition motion tried: Tru

In [63]:
pscene.set_object_state(initial_state)

In [64]:
schedules = ppline.tplan.find_schedules(False)
schedules_sorted = ppline.tplan.sort_schedule(schedules)
snode_schedule = ppline.tplan.idxSchedule2SnodeScedule(schedules_sorted[0])

In [65]:
ppline.play_schedule(snode_schedule=snode_schedule, period=0.01)

('table',)->('table',)
('table',)->('grip0',)
('grip0',)->('goal',)
('goal',)->('goal',)
